## Tutorial: 

### Configure the Environment

In [1]:
import json
import os
import chi,chi.ssh
from datetime import datetime, timedelta
from dateutil import tz

import chi

#Config with your project and site
chi.use_site('CHI@UC')
chi.set('project_name', 'CHI-231095')

# Tip: Name resources with your username for easier identification
username = os.getenv("USER")
prefix = username + "_k8s_ml_"
server_name = prefix + 'Server'
network_name = prefix + 'Net'
subnet_name = prefix + 'Subnet'
router_name = prefix + 'Router'
lease_name = prefix + 'Lease'

#Server attributes
image_name='CC-Ubuntu20.04'
node_type="compute_cascadelake_r"
server_count=3
physical_network="physnet1"
subnet_cidr = '192.168.1.0/24'


Now using CHI@UC:
URL: https://chi.uc.chameleoncloud.org
Location: Argonne National Laboratory, Lemont, Illinois, USA
Support contact: help@chameleoncloud.org


### Isolated Network

#### Create the Network

In [2]:
import chi.network

# Create a network out of provider network physnet1
network = chi.network.create_network(network_name, provider='physnet1')
network_id = network['id']

print(f'Network ID: {network_id}')

Network ID: 59437e98-8257-4d02-a0e1-594e8237f509


#### Configure the Network


In [3]:
subnet = chi.network.create_subnet(subnet_name, network_id, cidr=subnet_cidr, gateway_ip=None)
router = chi.network.create_router(router_name, gw_network_name='public')
chi.network.add_subnet_to_router(router['id'], subnet['id'])

{'id': '50393c69-455f-4539-b1f4-181b94efd0f0',
 'tenant_id': 'cb970a4b0f2e42c9b1b3f9015d02f8a5',
 'port_id': 'f0db5a76-1bd7-46e2-94e1-ae650014d762',
 'network_id': '59437e98-8257-4d02-a0e1-594e8237f509',
 'subnet_id': '0ab4302b-e723-43c3-a607-cecb3bf68160',
 'subnet_ids': ['0ab4302b-e723-43c3-a607-cecb3bf68160']}

### Servers on the Network



#### Create a Lease

In [4]:
import chi.lease

BLAZAR_TIME_FORMAT = '%Y-%m-%d %H:%M'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=7)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
chi.lease.add_node_reservation(reservation_list, count=server_count, node_type=node_type)
chi.lease.add_fip_reservation(reservation_list, count=server_count)

# Create the lease
lease = chi.lease.create_lease(lease_name, 
                               start_date=start_date,
                               end_date=end_date,
                               reservations=reservation_list)

lease_id = lease["id"]

chi.lease.wait_for_active(lease_id)

{'created_at': '2023-04-30 02:52:20',
 'updated_at': '2023-04-30 02:53:11',
 'id': 'f950d6a4-276d-4277-98aa-ad666e6eeccd',
 'name': 'cp3793_nyu_edu_k8s_ml_Lease',
 'user_id': '50de6f77f6d1941774ecf322c9f0ad2a3e2c128f4707c3278d98fd4a98b86d85',
 'project_id': 'cb970a4b0f2e42c9b1b3f9015d02f8a5',
 'start_date': '2023-04-30T02:53:00.000000',
 'end_date': '2023-05-07T02:52:00.000000',
 'trust_id': '1a303afc3bc6440eae1f7d07acb2bca3',
 'status': 'ACTIVE',
 'degraded': False,
 'reservations': [{'created_at': '2023-04-30 02:52:22',
   'updated_at': '2023-04-30 02:53:08',
   'id': '3d2afcff-9bcf-4714-bd28-8554546a6f7e',
   'lease_id': 'f950d6a4-276d-4277-98aa-ad666e6eeccd',
   'resource_id': '980575f8-dc21-4a75-ba20-2bc548042538',
   'resource_type': 'virtual:floatingip',
   'status': 'active',
   'missing_resources': False,
   'resources_changed': False,
   'network_id': '44b38c44-2a42-4b6d-b129-6c8f1b2a1375',
   'amount': 3,
   'required_floatingips': []},
  {'created_at': '2023-04-30 02:52:20'

#### Get the Reservations

In [5]:
compute_reservation_id = [reservation for reservation in lease['reservations'] if reservation['resource_type'] == 'physical:host'][0]['id']
floatingip_reservation_id = [reservation for reservation in lease['reservations'] if reservation['resource_type'] == 'virtual:floatingip'][0]['id']

print(f"compute_reservation_id: {compute_reservation_id}")
print(f"floatingip_reservation_id: {floatingip_reservation_id}")

compute_reservation_id: 9c2cff08-11c0-4ee9-a9e5-f07986988dc1
floatingip_reservation_id: 3d2afcff-9bcf-4714-bd28-8554546a6f7e


#### Start the Server

The `create_server` method won't work until your lease is active! Verify your lease is active by going to __Reservations__ > __Leases__ in your Chameleon project, and checking that the lease with your name has the status _ACTIVE_.

Don't worry if you ran it too early and your server (in __Compute__ > __Instances__) has the _Error_ status -- if this happens, you can just delete your instance using the Chameleon interface and run this cell again.

In [6]:
#create the server
server = chi.server.create_server(server_name, 
                                  compute_reservation_id,
                                  network_id=network_id,
                                  nics=[],
                                  image_name=image_name,
                                  count=3)


#### Associate the Floating IP


In [7]:
nodes = {}

for j,i in enumerate(server):
    chi.server.wait_for_active(i.id)
    floating_ip = chi.server.associate_floating_ip(i.id)
    nodes[j] = floating_ip

    print(f'Floating IP of node_{j}: {floating_ip}')

Floating IP of node_0: 192.5.86.204
Floating IP of node_1: 192.5.86.215
Floating IP of node_2: 192.5.86.213


#### Generate ssh key for each host

In [8]:
import chi, chi.ssh
for node_ip in nodes.values():
    chi.server.wait_for_tcp(node_ip, port=22)
    remote = chi.ssh.Remote(node_ip)
    remote.run('ssh-keygen -t rsa -b 4096 -f ~/.ssh/id_rsa -q -N ""')

/opt/conda/lib/python3.10/site-packages/paramiko/client.py:835: UserWarning: Unknown ssh-ed25519 host key for 192.5.86.204: b'7f2fcdba31886a7058e54d62436cfb26'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/paramiko/client.py:835: UserWarning: Unknown ssh-ed25519 host key for 192.5.86.215: b'ddfe6c28da87862f2aa608d6efed2a2a'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/paramiko/client.py:835: UserWarning: Unknown ssh-ed25519 host key for 192.5.86.213: b'9996a49251441c1117e894e35570ad0c'
  warnings.warn(


#### Key Transfer
Add each hosts public key to the authorized_keys file located in ~/.ssh folder in every host to make sure that all the hosts are able to ssh into each other.

In [9]:
for node_ip in nodes.values():
    chi.server.wait_for_tcp(node_ip, port=22)
    remote = chi.ssh.Remote(node_ip)
    public_key = remote.run('cat ~/.ssh/id_rsa.pub').tail("stdout")[2:]
    public_key = public_key.replace("\n", "")
    for node_ip in nodes.values():
        remote = chi.ssh.Remote(node_ip)
        remote.run(f'echo {public_key} >> ~/.ssh/authorized_keys') 

ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQCl3HP4QLGTunDBdx4FYGMDwQEZlX7xFCxIbepK1aMTeOoTrbo3SpiCMpzUMcZBoNHSf5Tu9Qh1dMhz7jNDNK4UeiGc9FFdItPG2Z9Te62lze9NwIsWjbVNgKjOWRSP3UUnBqfjAsOcsWPXZzz1KmBR46+XjE3pBIdPpShY8ASj4R28L8F8au3KC/u3FKbCZ5W3eBnvZZwaR2xg3GYzLLw5eT/3oU2MYDj0MgX/mC2yiWT92rrZQHdIfxdFIGJvwYoMfz3WBhd5NP3YVOQbHXSCw8iD18kR5GDTR0hnUoviGPpMvHfjj+vceS3pZzrsvNp8cmAMdPxcF3jYERXVYd/NLIPqZGByHW2JHn5Aow/MErONv+N8zrD+UShmAS08KWR7IaZBcEmzSGca7dI4m4itES/5OjAJGP79MqGJKGORze1WP/ZA6geQXFFOFODl50s8MzRmE3i48GUzYH3xuNfRcyO1jvhkEvBR/DfgYaSM49T38sxFNl2h7LzNn6vEaWEwaQnF2hscYKGXnAqT4fiTLKmTLq456vYr3v2QwgWOp07s2TxSfd0QiTapvtExNEt0jb5TVGv/e2bgnBB7iOyNVCa9URkVs9JCdrZ7dUhicjc/LfEYjdP4kLLtGhLFej2bEFwZuboUojq3+viD7Hm7ru6yj4vTrSEezj94lutt7w== cc@cp3793-nyu-edu-k8s-ml-server-3
ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDIpGDY0Zi2CTmGLNmgOl8ZCWb+MKRDHZkMCgWOAlgmAl9eGaGax97Lw9TNMe/2ehTDYCPhbQAdeL8h+aNCieNhnqwjYGYQAVNemGznEnPJ3QXUFD15SKAqq/SCTLCvQx+RCEoaGRIBVO3qVAnnvyzJHdPNmjZkoILIdivKVzDv66SLDLRO8y69UpeSKlEqAqb/gwca2pmqI

#### Store the physical ip of each host
During the process of creating the cluster we will be needing the physical ips of each host so this will be usefull.

In [10]:
physical_ips=[]
for node_ip in nodes.values():
    chi.server.wait_for_tcp(node_ip, port=22)
    remote = chi.ssh.Remote(node_ip)
    physical_ips.append(remote.run('hostname -I').tail("stdout")[2:-1])

192.168.1.172 
192.168.1.189 
192.168.1.156 


#### setting up the cluster

In [11]:
ansible_node = nodes[1]
remote = chi.ssh.Remote(ansible_node)
remote.run("git clone https://github.com/kubernetes-sigs/kubespray.git")
remote.run("cd kubespray; git checkout release-2.16")
remote.run("cd kubespray; sudo pip3 install -r requirements.txt")
remote.run("cd kubespray; cp -rfp inventory/sample inventory/mycluster")
remote.run(f"declare -a IPS=({physical_ips[0]} {physical_ips[1]} {physical_ips[2]});"+"cd kubespray; CONFIG_FILE=inventory/mycluster/hosts.yaml python3 contrib/inventory_builder/inventory.py ${IPS[@]}")

Cloning into 'kubespray'...
Switched to a new branch 'release-2.16'


Branch 'release-2.16' set up to track remote branch 'release-2.16' from 'origin'.
  Created wheel for ansible: filename=ansible-2.9.20-py3-none-any.whl size=16203441 sha256=96fb47801424788e09a65dbb64f61d559ad1d3d0da67352f0a20e4d0d4fe14c2
  Stored in directory: /root/.cache/pip/wheels/61/a8/07/9b4017c5cbc3e015ecb6cbc5b37aa974cb485cae9d46bc6728
Successfully built ansible
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 1.1.0
    Not uninstalling markupsafe at /usr/lib/python3/dist-packages, outside environment /usr
    Can't uninstall 'MarkupSafe'. No files were found to uninstall.
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.10.1
    Not uninstalling jinja2 at /usr/lib/python3/dist-packages, outside environment /usr
    Can't uninstall 'Jinja2'. No files were found to uninstall.
  Attempting uninstall: ansible
    Found existing installation: ansible 2.9.6
    Not uninstalling ansible at /usr/lib/python3/dist-packages, outside en

<Result cmd='declare -a IPS=(192.168.1.172 192.168.1.189 192.168.1.156);cd kubespray; CONFIG_FILE=inventory/mycluster/hosts.yaml python3 contrib/inventory_builder/inventory.py ${IPS[@]}' exited=0>

In [13]:
remote.run("cd kubespray; cat inventory/mycluster/group_vars/all/all.yml")

---
## Directory where etcd data stored
etcd_data_dir: /var/lib/etcd

## Experimental kubeadm etcd deployment mode. Available only for new deployment
etcd_kubeadm_enabled: false

## Directory where the binaries will be installed
bin_dir: /usr/local/bin

## The access_ip variable is used to define how other nodes should access
## the node.  This is used in flannel to allow other flannel nodes to see
## this node for example.  The access_ip is really useful AWS and Google
## environments where the nodes are accessed remotely by the "public" ip,
## but don't know about that address themselves.
# access_ip: 1.1.1.1


## External LB example config
## apiserver_loadbalancer_domain_name: "elb.some.domain"
# loadbalancer_apiserver:
#   address: 1.2.3.4
#   port: 1234

## Internal loadbalancers for apiservers
# loadbalancer_apiserver_localhost: true
# valid options are "nginx" or "haproxy"
# loadbalancer_apiserver_type: nginx  # valid values "nginx" or "haproxy"

## If the cilium is going to be

<Result cmd='cd kubespray; cat inventory/mycluster/group_vars/all/all.yml' exited=0>

In [16]:
remote.run("cat kubespray/inventory/mycluster/group_vars/k8s_cluster/k8s-cluster.yml")

---
# Kubernetes configuration dirs and system namespace.
# Those are where all the additional config stuff goes
# the kubernetes normally puts in /srv/kubernetes.
# This puts them in a sane location and namespace.
# Editing those values will almost surely break something.
kube_config_dir: /etc/kubernetes
kube_script_dir: "{{ bin_dir }}/kubernetes-scripts"
kube_manifest_dir: "{{ kube_config_dir }}/manifests"

# This is where all the cert scripts and certs will be located
kube_cert_dir: "{{ kube_config_dir }}/ssl"

# This is where all of the bearer tokens will be stored
kube_token_dir: "{{ kube_config_dir }}/tokens"

kube_api_anonymous_auth: true

## Change this to use another Kubernetes version, e.g. a current beta release
kube_version: v1.20.7

# Where the binaries will be downloaded.
# Note: ensure that you've enough disk space (about 1G)
local_release_dir: "/tmp/releases"
# Random shifts for retrying failed ops like pushing/downloading
retry_stagger: 5

# This is the group that the 

<Result cmd='cat kubespray/inventory/mycluster/group_vars/k8s_cluster/k8s-cluster.yml' exited=0>

In [17]:
remote.run("cat kubespray/inventory/mycluster/group_vars/k8s_cluster/addons.yml")

---
# Kubernetes dashboard
# RBAC required. see docs/getting-started.md for access details.
# dashboard_enabled: false

# Helm deployment
helm_enabled: false

# Registry deployment
registry_enabled: false
# registry_namespace: kube-system
# registry_storage_class: ""
# registry_disk_size: "10Gi"

# Metrics Server deployment
metrics_server_enabled: false
# metrics_server_kubelet_insecure_tls: true
# metrics_server_metric_resolution: 60s
# metrics_server_kubelet_preferred_address_types: "InternalIP"

# Rancher Local Path Provisioner
local_path_provisioner_enabled: false
# local_path_provisioner_namespace: "local-path-storage"
# local_path_provisioner_storage_class: "local-path"
# local_path_provisioner_reclaim_policy: Delete
# local_path_provisioner_claim_root: /opt/local-path-provisioner/
# local_path_provisioner_debug: false
# local_path_provisioner_image_repo: "rancher/local-path-provisioner"
# local_path_provisioner_image_tag: "v0.0.19"
# local_path_provisioner_helper_image_repo: "bu

<Result cmd='cat kubespray/inventory/mycluster/group_vars/k8s_cluster/addons.yml' exited=0>

In [18]:
remote.run("cd kubespray; cat inventory/mycluster/group_vars/all/docker.yml")

---
## Uncomment this if you want to force overlay/overlay2 as docker storage driver
## Please note that overlay2 is only supported on newer kernels
# docker_storage_options: -s overlay2

## Enable docker_container_storage_setup, it will configure devicemapper driver on Centos7 or RedHat7.
docker_container_storage_setup: false

## It must be define a disk path for docker_container_storage_setup_devs.
## Otherwise docker-storage-setup will be executed incorrectly.
# docker_container_storage_setup_devs: /dev/vdb

## Uncomment this if you want to change the Docker Cgroup driver (native.cgroupdriver)
## Valid options are systemd or cgroupfs, default is systemd
# docker_cgroup_driver: systemd

## Only set this if you have more than 3 nameservers:
## If true Kubespray will only use the first 3, otherwise it will fail
docker_dns_servers_strict: false

# Path used to store Docker data
docker_daemon_graph: "/var/lib/docker"

## Used to set docker daemon iptables options to true
docker_iptables_

<Result cmd='cd kubespray; cat inventory/mycluster/group_vars/all/docker.yml' exited=0>

#### changed yml files

In [19]:
remote.run("cd kubespray; cat inventory/mycluster/group_vars/all/all.yml")

---
## Directory where etcd data stored
etcd_data_dir: /var/lib/etcd

## Experimental kubeadm etcd deployment mode. Available only for new deployment
etcd_kubeadm_enabled: false

## Directory where the binaries will be installed
bin_dir: /usr/local/bin

## The access_ip variable is used to define how other nodes should access
## the node.  This is used in flannel to allow other flannel nodes to see
## this node for example.  The access_ip is really useful AWS and Google
## environments where the nodes are accessed remotely by the "public" ip,
## but don't know about that address themselves.
# access_ip: 1.1.1.1


## External LB example config
## apiserver_loadbalancer_domain_name: "elb.some.domain"
# loadbalancer_apiserver:
#   address: 1.2.3.4
#   port: 1234

## Internal loadbalancers for apiservers
# loadbalancer_apiserver_localhost: true
# valid options are "nginx" or "haproxy"
# loadbalancer_apiserver_type: nginx  # valid values "nginx" or "haproxy"

## If the cilium is going to be

<Result cmd='cd kubespray; cat inventory/mycluster/group_vars/all/all.yml' exited=0>

In [20]:
remote.run("cd kubespray; cat inventory/mycluster/group_vars/k8s_cluster/k8s-cluster.yml")

---
# Kubernetes configuration dirs and system namespace.
# Those are where all the additional config stuff goes
# the kubernetes normally puts in /srv/kubernetes.
# This puts them in a sane location and namespace.
# Editing those values will almost surely break something.
kube_config_dir: /etc/kubernetes
kube_script_dir: "{{ bin_dir }}/kubernetes-scripts"
kube_manifest_dir: "{{ kube_config_dir }}/manifests"

# This is where all the cert scripts and certs will be located
kube_cert_dir: "{{ kube_config_dir }}/ssl"

# This is where all of the bearer tokens will be stored
kube_token_dir: "{{ kube_config_dir }}/tokens"

kube_api_anonymous_auth: true

## Change this to use another Kubernetes version, e.g. a current beta release
kube_version: v1.20.7

# Where the binaries will be downloaded.
# Note: ensure that you've enough disk space (about 1G)
local_release_dir: "/tmp/releases"
# Random shifts for retrying failed ops like pushing/downloading
retry_stagger: 5

# This is the group that the 

<Result cmd='cd kubespray; cat inventory/mycluster/group_vars/k8s_cluster/k8s-cluster.yml' exited=0>

In [21]:
remote.run("cat kubespray/inventory/mycluster/group_vars/k8s_cluster/addons.yml")

---
# Kubernetes dashboard
# RBAC required. see docs/getting-started.md for access details.
# dashboard_enabled: false

# Helm deployment
helm_enabled: true
helm_stable_repo_url: "https://charts.helm.sh/stable"
# Registry deployment
registry_enabled: false
# registry_namespace: kube-system
# registry_storage_class: ""
# registry_disk_size: "10Gi"

# Metrics Server deployment
metrics_server_enabled: false
# metrics_server_kubelet_insecure_tls: true
# metrics_server_metric_resolution: 60s
# metrics_server_kubelet_preferred_address_types: "InternalIP"

# Rancher Local Path Provisioner
local_path_provisioner_enabled: false
# local_path_provisioner_namespace: "local-path-storage"
# local_path_provisioner_storage_class: "local-path"
# local_path_provisioner_reclaim_policy: Delete
# local_path_provisioner_claim_root: /opt/local-path-provisioner/
# local_path_provisioner_debug: false
# local_path_provisioner_image_repo: "rancher/local-path-provisioner"
# local_path_provisioner_image_tag: "v0.0

<Result cmd='cat kubespray/inventory/mycluster/group_vars/k8s_cluster/addons.yml' exited=0>

In [22]:
remote.run("cd kubespray; cat inventory/mycluster/group_vars/all/docker.yml")

---
## Uncomment this if you want to force overlay/overlay2 as docker storage driver
## Please note that overlay2 is only supported on newer kernels
# docker_storage_options: -s overlay2

## Enable docker_container_storage_setup, it will configure devicemapper driver on Centos7 or RedHat7.
docker_container_storage_setup: false

## It must be define a disk path for docker_container_storage_setup_devs.
## Otherwise docker-storage-setup will be executed incorrectly.
# docker_container_storage_setup_devs: /dev/vdb

## Uncomment this if you want to change the Docker Cgroup driver (native.cgroupdriver)
## Valid options are systemd or cgroupfs, default is systemd
# docker_cgroup_driver: systemd

## Only set this if you have more than 3 nameservers:
## If true Kubespray will only use the first 3, otherwise it will fail
docker_dns_servers_strict: false

# Path used to store Docker data
docker_daemon_graph: "/var/lib/docker"

## Used to set docker daemon iptables options to true
docker_iptables_

<Result cmd='cd kubespray; cat inventory/mycluster/group_vars/all/docker.yml' exited=0>

In [23]:
remote.run("cd kubespray; ansible-playbook -i inventory/mycluster/hosts.yaml  --become --become-user=root cluster.yml")


PLAY [localhost] ***************************************************************
Sunday 30 April 2023  04:40:43 +0000 (0:00:00.040)       0:00:00.040 ********** 

TASK [Check 2.9.0 <= Ansible version < 2.11.0] *********************************
ok: [localhost] => {
    "changed": false,
    "msg": "All assertions passed"
}
Sunday 30 April 2023  04:40:43 +0000 (0:00:00.027)       0:00:00.067 ********** 

TASK [Check that python netaddr is installed] **********************************
ok: [localhost] => {
    "changed": false,
    "msg": "All assertions passed"
}
Sunday 30 April 2023  04:40:43 +0000 (0:00:00.014)       0:00:00.081 ********** 

TASK [Check that jinja is not too old (install via pip)] ***********************
ok: [localhost] => {
    "changed": false,
    "msg": "All assertions passed"
}


[WARNING]: Could not match supplied host pattern, ignoring: kube-master



PLAY [Add kube-master nodes to kube_control_plane] *****************************
skipping: no hosts matched


[WARNING]: Could not match supplied host pattern, ignoring: kube-node
[WARNING]: Could not match supplied host pattern, ignoring: k8s-cluster



PLAY [Add kube-node nodes to kube_node] ****************************************
skipping: no hosts matched


...

[WARNING]: flush_handlers task does not support when conditional


Sunday 30 April 2023  04:42:21 +0000 (0:00:00.041)       0:01:37.685 ********** 
Sunday 30 April 2023  04:42:21 +0000 (0:00:00.035)       0:01:37.721 ********** 

TASK [container-engine/containerd-common : gather os specific variables] *******
ok: [node1] => (item=/home/cc/kubespray/roles/container-engine/containerd-common/vars/../vars/ubuntu.yml)
ok: [node2] => (item=/home/cc/kubespray/roles/container-engine/containerd-common/vars/../vars/ubuntu.yml)
ok: [node3] => (item=/home/cc/kubespray/roles/container-engine/containerd-common/vars/../vars/ubuntu.yml)
Sunday 30 April 2023  04:42:21 +0000 (0:00:00.061)       0:01:37.783 ********** 
Sunday 30 April 2023  04:42:21 +0000 (0:00:00.050)       0:01:37.833 ********** 
Sunday 30 April 2023  04:42:21 +0000 (0:00:00.040)       0:01:37.874 ********** 
Sunday 30 April 2023  04:42:21 +0000 (0:00:00.034)       0:01:37.908 ********** 
Sunday 30 April 2023  04:42:21 +0000 (0:00:00.036)       0:01:37.945 ********** 
Sunday 30 April 2023  04:42:21 +0

[WARNING]: noop task does not support when conditional



TASK [download : prep_kubeadm_images | Create kubeadm config] ******************
changed: [node1]
changed: [node2]
Sunday 30 April 2023  04:43:01 +0000 (0:00:00.451)       0:02:18.488 ********** 

TASK [prep_kubeadm_images | Copy kubeadm binary from download dir to system path] ***
changed: [node1]
changed: [node2]
Sunday 30 April 2023  04:43:02 +0000 (0:00:00.431)       0:02:18.919 ********** 

TASK [download : prep_kubeadm_images | Set kubeadm binary permissions] *********
ok: [node1]
ok: [node2]
Sunday 30 April 2023  04:43:02 +0000 (0:00:00.185)       0:02:19.105 ********** 

TASK [download : prep_kubeadm_images | Generate list of required images] *******
ok: [node1]
Sunday 30 April 2023  04:43:02 +0000 (0:00:00.333)       0:02:19.438 ********** 

TASK [download : prep_kubeadm_images | Parse list of images] *******************
ok: [node1] => (item=k8s.gcr.io/kube-apiserver:v1.20.7)
ok: [node1] => (item=k8s.gcr.io/kube-controller-manager:v1.20.7)
ok: [node1] => (item=k8s.gcr.io/kube

<Result cmd='cd kubespray; ansible-playbook -i inventory/mycluster/hosts.yaml  --become --become-user=root cluster.yml' exited=0>

In [2]:

import chi, chi.ssh
remote = chi.ssh.Remote("192.5.86.215")

In [3]:
remote.run("git clone https://github.com/teaching-on-testbeds/k8s-ml.git")

/opt/conda/lib/python3.10/site-packages/paramiko/client.py:835: UserWarning: Unknown ssh-ed25519 host key for 192.5.86.215: b'ddfe6c28da87862f2aa608d6efed2a2a'
  warnings.warn(
Cloning into 'k8s-ml'...


<Result cmd='git clone https://github.com/teaching-on-testbeds/k8s-ml.git' exited=0>

In [5]:
remote.run("cd k8s-ml; cd app; docker build -t ml-app .")

Sending build context to Docker daemon  461.3kB
Step 1/11 : FROM python:3.9-slim-buster
3.9-slim-buster: Pulling from library/python
9fbefa337077: Pulling fs layer
a25702e0699e: Pulling fs layer
176f9dba58d6: Pulling fs layer
7060fb9d5689: Pulling fs layer
37ec3c889b82: Pulling fs layer
7060fb9d5689: Waiting
a25702e0699e: Verifying Checksum
7060fb9d5689: Download complete
176f9dba58d6: Verifying Checksum
176f9dba58d6: Download complete
9fbefa337077: Verifying Checksum
9fbefa337077: Download complete
37ec3c889b82: Verifying Checksum
37ec3c889b82: Download complete
9fbefa337077: Pull complete
a25702e0699e: Pull complete
176f9dba58d6: Pull complete
7060fb9d5689: Pull complete
37ec3c889b82: Pull complete
Digest: sha256:1c5091a9ba3001c8e8182f56a740f26e5b328b7aa3f2c1f7d974e2e05282f323
Status: Downloaded newer image for python:3.9-slim-buster
 ---> b3a7bf671c0f
Step 2/11 : RUN pip install virtualenv
 ---> Running in ed5e29bb1d85
ERROR: Could not find a version that satisfies the requirement v

The command '/bin/sh -c pip install virtualenv' returned a non-zero code: 1


UnexpectedExit: Encountered a bad command exit code!

Command: 'cd k8s-ml; cd app; docker build -t ml-app .'

Exit code: 1

Stdout: already printed

Stderr: already printed



#### Delete Server

In [ ]:
for i in server:
  chi.server.delete_server(i.id)

#### De-configure Network



In [ ]:
router_id = router['id']
subnet_id = subnet['id']

try:
    result = chi.network.remove_subnet_from_router(router_id, subnet_id)
except Exception as e:
    print(f"detach_router_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_router(router_id)
except Exception as e:
    print(f"delete_router_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_subnet(subnet_id)
except Exception as e:
    print(f"delete_subnet_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_network(network_id)
except Exception as e:
    print(f"delete_network_by_name error: {str(e)}")
    pass


#### Release Lease

In [ ]:
chi.lease.delete_lease(lease_id)